In [2]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.special import log_ndtr


n = 10
m = 45
m_test = 45
sigma = 0.25

train = np.array([
    [1, 2, 1],
    [1, 3, 1],
    [1, 4, 1],
    [1, 5, 1],
    [1, 6, 1],
    [1, 7, 1],
    [1, 8, 1],
    [1, 9, 1],
    [1, 10, 1],
    [2, 3, -1],
    [2, 4, -1],
    [2, 5, -1],
    [2, 6, -1],
    [2, 7, -1],
    [2, 8, -1],
    [2, 9, -1],
    [2, 10, -1],
    [3, 4, 1],
    [3, 5, -1],
    [3, 6, -1],
    [3, 7, 1],
    [3, 8, 1],
    [3, 9, 1],
    [3, 10, 1],
    [4, 5, -1],
    [4, 6, -1],
    [4, 7, 1],
    [4, 8, 1],
    [4, 9, -1],
    [4, 10, -1],
    [5, 6, 1],
    [5, 7, 1],
    [5, 8, 1],
    [5, 9, -1],
    [5, 10, 1],
    [6, 7, 1],
    [6, 8, 1],
    [6, 9, -1],
    [6, 10, -1],
    [7, 8, 1],
    [7, 9, 1],
    [7, 10, -1],
    [8, 9, -1],
    [8, 10, -1],
    [9, 10, 1]
])

test = np.array([[1, 2, 1],
                 [1, 3, 1],
                 [1, 4, 1],
                 [1, 5, 1],
                 [1, 6, 1],
                 [1, 7, 1],
                 [1, 8, 1],
                 [1, 9, 1],
                 [1, 10, 1],
                 [2, 3, -1],
                 [2, 4, 1],
                 [2, 5, -1],
                 [2, 6, -1],
                 [2, 7, -1],
                 [2, 8, 1],
                 [2, 9, -1],
                 [2, 10, -1],
                 [3, 4, 1],
                 [3, 5, -1],
                 [3, 6, 1],
                 [3, 7, 1],
                 [3, 8, 1],
                 [3, 9, -1],
                 [3, 10, 1],
                 [4, 5, -1],
                 [4, 6, -1],
                 [4, 7, -1],
                 [4, 8, 1],
                 [4, 9, -1],
                 [4, 10, -1],
                 [5, 6, -1],
                 [5, 7, 1],
                 [5, 8, 1],
                 [5, 9, 1],
                 [5, 10, 1],
                 [6, 7, 1],
                 [6, 8, 1],
                 [6, 9, 1],
                 [6, 10, 1],
                 [7, 8, 1],
                 [7, 9, -1],
                 [7, 10, 1],
                 [8, 9, -1],
                 [8, 10, -1],
                 [9, 10, 1]])

In [12]:
np.ones([45,])

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [15]:
import scipy.sparse as sp
import cvxpy as cp
from scipy.special import log_ndtr

            

# Form adjacency matrix
A1 = sp.coo_matrix((train[:, 2], (range(m), train[:, 0]-np.ones([45,]))), shape=(m, n))
A2 = sp.coo_matrix((train[:, 2], (range(m), train[:, 1]-np.ones([45,]))), shape=(m, n))
A = (A1 - A2).toarray()

# Estimate abilities
a_hat = cp.Variable(n)
objective = cp.Minimize(-cp.sum(cp.log_normcdf(A @ a_hat / sigma)))
constraints = [0 <= a_hat, a_hat <= 1]
problem = cp.Problem(objective, constraints)
problem.solve()

11.430294646964965

In [17]:
a=a_hat.value
a

array([ 1.00000000e+00, -3.07024986e-21,  6.78273774e-01,  3.68704618e-01,
        7.90021663e-01,  5.81306405e-01,  3.87382448e-01,  8.54414113e-02,
        6.70046253e-01,  5.77486181e-01])

In [19]:
import scipy.sparse as sp
import numpy as np

# Sparse matrix A_test
A1_test = sp.csr_matrix((np.ones(m_test), (range(m_test), test[:, 0]-1)), shape=(m_test, n))
A2_test = sp.csr_matrix((np.ones(m_test)*-1, (range(m_test), test[:, 1]-1)), shape=(m_test, n))
A_test = A1_test + A2_test

# Prediction and error rate calculation
res = np.sign(A_test.dot(a_hat.value))
Pml = 1 - np.count_nonzero(res - test[:, 2]) / m_test
Ply = 1 - np.count_nonzero(train[:, 2] - test[:, 2]) / m_test

In [21]:
Pml,Ply

(0.8666666666666667, 0.7555555555555555)